# Facility Location in Disaster Relief Operations

<div style="text-align: justify"> A facility location model to support humanitarian logistics operations was designed using Mixed Integer Linear Programming, in which a finite number of demand nodes could be satisfied by a set of supply nodes, considering not only the costs related to these locations but also constraints aimed at complying with timely humanitarian attention in an operation of this type. Moreover, an integrated solution scheme was proposed, which includes the use of a GeoJSON API, along with a translation of the model into Python language statements for its solution using Gurobi. Subsequently, a case study was executed analyzing the operation of the water supply through water tanks during a natural disaster, which occurred during 2017 in Mocoa, Colombia. The results reflected positive effects on the prioritization of the average distance traveled and on the satisfaction of shelter demand, along with the costs associated with the implementation of the optimal solution.</div>

### Problem Description:
&nbsp;
<div style="text-align: justify">
    The proposed model was implemented within the context of the natural disaster that occurred in the municipality of Mocoa, in Colombia. This event occurred on the night of March 31, 2017, due to the sudden overflow of the Mocoa, Sangoyaco, and Mulato rivers, and the Taruca, Taruquilla, and La Mision streams. This was followed by an avalanche that seriously impacted this municipality, leaving in its wake not only reports of the dead, injured, and missing people but also a suspension of basic services. During this emergency, water services were suspended, and demand was met with the help of water tanks in various sectors of the municipality. The proposed model was implemented to find the optimal configuration of the location of the water supply tanks to meet the demand of the shelters enabled, as it was possible to access accurate data of this operation. Thus, the goal was to find the locations of these tanks that would minimize the associated costs and, at the same time, improve response time. The supply tanks used were of four types, defined according to their capacity in liters: 250 L, 5,000 L, 10,000 L, and 20,000 L. A total of 35 candidate locations were considered. In addition, because each location had the possibility of establishing a tank of any of the four types, a total of 140 location possibilities were considered. Also, the guidelines established for this type of operation were analyzed in order to include the demand for water from the shelters. According to the National Shelter Management Manual of the Colombian Red Cross Society, the community must be guaranteed a minimum of 15 liters of water per inhabitant per day to carry out basic activities.</div>
&nbsp;
<div style="text-align: justify">As of the date of the report, 12 shelters had been opened serving nearly 726 families, for a total of 2462 people approximately. Regarding the objective function, fixed costs were determined by the unit value of establishing a water supply tank of a certain capacity. The variable cost corresponded to the transportation cost incurred in the supply operation. The water treatment plants keep the supply in tank cars that supply water tanks, the value was calculated according to the liters of water transported. For example, transporting 1000 gallons of water cost approximately 16.38 USD. Since the problem included 12 shelters and 140 possible locations for the tanks, a GeoJSON API was applied to calculate these 1680 distance combinations using Google Maps. The exact geographical coordinates of the nodes were considered in the model in order to achieve a realistic solution. Concerning the entry data of constraints related to short response time, the maximum allowed distance was set at 0.5 km, then the weighted average distance was less than this value. Similarly, the minimum permissible demand within this distance was required to be greater than or equal to 85%. This was based on the guidelines of the National Manual for Shelter Management of the Colombian Red Cross Society, which states that the distance between any shelter and the nearest place of supply must not exceed 500 meters. Also, for the attention limit constraint, the manual indicated a maximum of 250 people per water source, which is equivalent to a supply limit of 3750 liters. Thus, people do not have to wait too long to fill their containers.</div>

### Installation - Google Maps Services:

In order to run this Jupyter Notebook properly, you must have Google Maps Services Run the next cell to install. Also, you need a API KEY for Google Maps Platform. 

In [ ]:
pip install -U googlemaps

### Distance Matrix:
<div style="text-align: justify">Compute the distance between combinations of supply points and shelters is an input required by the model. In addition, a network structure defined on maps and routing optimization algorithms is required. However, the availability of this data and the price of adequate mapping can be a challenge. To overcome these limitations, a Python code was written, integrated with the GeoJSON API. With this code, it is possible to collect real distance data from the road network for any case study and record this information in a spreadsheet ('distance.xlsx'). Web mapping services are web services that allow viewing, querying, and manipulating or processing geospatial maps and/or data. In this case, the API distance matrix created by Google is a service that provides the distance and travel time for a matrix of origins ('supply_points.xlsx') and destinations('shelters.xlsx'), according to the recommended route between the start and endpoints.</div>

In [ ]:
import googlemaps
import pandas as pd

gmaps = googlemaps.Client(key='INSERT YOUR API KEY HERE')
supply_points = pd.read_excel('supply_points.xlsx')
shelters = pd.read_excel('shelters.xlsx')

point = shelters['POINT'].tolist()

data = ['ID_SUPPLY_POINT', 'SUPPLY_POINT']
data += point
matrix = pd.DataFrame(columns=data)

for index, supply_point in supply_points.iterrows():
    info_distance = []
    for index, shelter in shelters.iterrows():
        try:
            calculate_distance = gmaps.distance_matrix(supply_point['LOCATION'], shelter['LOCATION'])
            distance = (calculate_distance['rows'][0]['elements'][0]['distance']['value']/1000)
        except:
              distance = ''
        info_distance.append(distance)
        
    info_temp = [supply_point['ID'], supply_point['LOCATION']]
    info_temp.extend(info_distance)
    temp = pd.DataFrame([info_temp], columns=data)
    matrix = matrix.append(temp, ignore_index=True)
                
new_file = pd.ExcelWriter('distance.xlsx')
matrix.to_excel(new_file, 'DATA', index=False)
new_file.save()

### Python Implementation

##### Gurobi License
In order to run this Jupyter Notebook properly, you must have a Gurobi license.

In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import *

#### Set and Indices:
Supply point $i$; Shelter $j$

$c_{ij}$: Cost of serving the shelter $j$ from the supply point $i$.

$C_{i}$: Fixed cost for opening a supply point $i$.

$S_{i}$: Capacity available at the supply point $i$. 

$D_{j}$: Demand from shelter $j$.

$W_{ij}$: A big number.

$d_{ij}$: Distance from the supply point $i$ to the shelter $j$.

$φ$: Maximum allowed average distance.

$ϕ$: Minimum demand satisfied within a distance value.

$δ$: Maximum attention flow.

$a_{ij}$: 1 if the distance of the supply point $i$ to $j$ is less than or equal to at a predetermined distance value, 0 otherwise.

$F_{min}$: Minimum number of supply points.

$F_{max}$: Maximum number of supply points.

In [ ]:
# We define all the input data for the model

df1 = pd.read_excel('shelters.xlsx', 'Sheet1', index_col=0) #Shelters
df2 = pd.read_excel('supply_points.xlsx', 'Sheet1', index_col=0) #Supply points
df3 = pd.read_excel('distance.xlsx', 'Sheet1', index_col=0) #Distance matrix

# Create dictionaries to capture supply limits and shelters demand:
id_shelters = list(df1.index)
id_supply_points = list(df2.index)
id_distance = list(df3.index)

arcs = [(i,j) for i in id_supply_points for j in id_shelters]

D_j = df1['Demand']
S_i = df2['Supply']
C_i = df2['Fixcost']
c_ij = df2['Varcost']

dist = {(id_supply_points[j],id_shelters[i]):df3.iloc[j][i+1] for j in range(len(id_supply_points)) for i in range(len(id_shelters))}

#### Model Deployment

In [ ]:
model = gp.Model('FacilityLocation')

#### Decision Variables:

$x_{ij}$: Flow in the arc of the supply point $i$ to the shelter $j$. $x_{ij}\geq0\ \ \ \forall ij$

$B_{i}$: 1 if the supply point $i$ is open; 0 otherwise.

In [ ]:
#Variables de decision

B_i = model.addVars(id_supply_points, name='B_i', vtype=GRB.BINARY)
x_ij = model.addVars(arcs, name='x_ij', vtype=GRB.INTEGER)

#### Objetive Function:

First, indexes $i$ and $j$ were defined, denoting a specific supply point and a shelter, respectively. The arc of a node $i$ to a node $j$ is represented by the notation $x_{ij}$. In addition, there is a total of $n$ supply points y $m$ shelters. The objective function $(z)$ aims to minimize total costs. The two most important components in making the decision to open new facilities were considered: a fixed cost $C_{i}$ incurred by locating or “opening” a supply point $B_{i}$ and a variable cost $c_{ij}$ depending on the ﬂow through the arc. For example, $c_{ij}$ could be characterized by a transportation cost that depends on the number of emergency kits that are being delivered to a shelter. Then, this function assumes for all possible arcs, a particular cost multiplied by the flow through $x_{ij}$:

$Min z = ∑_{i=1}^{n}∑_{j=1}^{m} c_{ij}x_{ij} + ∑_{i=1}^{n} C_{i}B_{i}$

In [ ]:
# funcion objetivo

z = quicksum(x_ij[i,j]*c_ij[i] for i in id_supply_points for j in id_shelters) + quicksum(B_i[i]*C_i[i] for i in id_supply_points)

model.setObjective(z, GRB.MINIMIZE)

A supply constraint was required. That is making sure that everything from the supply points does not exceed what is available there for all the $i$ points within the supply set ($S_{i}$):

$\sum_{j=1}^{m}x_{ij}\le S_{i}\ \ \ \ \ \ \forall i\in S$

In [ ]:
# restricciones
model.addConstrs(quicksum(x_ij[i,j] for j in id_shelters) <= S_i[i] for i in  id_supply_points)

In the same way, we need a constraint that makes sure that all the flows coming into a certain $j$ point must exceed or be equal to the demand at each one of those ($D_{j}$):

$\sum_{i=1}^n x_{ij} \geq D_{j}\ \ \ \ \ \ \ \forall j\in D$

In [ ]:
model.addConstrs(quicksum(x_ij[i,j] for i in id_supply_points)>= D_j[j] for j in id_shelters)

In addition, link constraints were considered. It is not possible to supply from a supply point unless it is open. $W_{ij}$ is a big number and the binary variable $B_{i}$ can take two values, either 0 or 1. $W_{ij}$ was proposed to set it equal to the sum of all the demand because the flow on each individual arc can never be greater than the total demand required.

$x_{ij}-W_{ij}B_i\le0\ \ \forall ji$

In [ ]:
W = D_j.sum()
model.addConstrs(x_ij[i,j]-D_j[j]*B_i[i] <= 0 for i in id_supply_points for j in id_shelters)

Although in this type of operation the economic factor takes second place to other aspects, it cannot be denied that it is implicit, because in any type of design of a supply chain a balance must be achieved between the costs and the services offered. On the other hand, the service is closely related to a short response time: humanitarian aid organizations want to ensure that the affected population is effectively served and want to mitigate the impact of the disaster and its consequences as soon as possible. Therefore, in general terms, the goal of this model was to find the scenario that implies the lowest costs and at the same time considers timely humanitarian assistance. However, the term "timely humanitarian assistance" could mean different things; considering that this problem focused on a strategic design to support humanitarian logistics operations, the model used distance as the most appropriate variable to achieve the proposed goals. This is based on the fact that the proximity between a supply point and a shelter is one of the best ways to ensure rapid response time and comprehensive care. Thus, the model includes the following two constraints:

A constraint for the maximum allowed average distance was included. Each $x_{ij}$ is multiplied by its distance $d_{ij}$ which corresponds to the distance from a supply point $i$ to a shelter $j$. This operation involves the calculation of a weighted distance. Also, this value is divided by the sum of the total demand ($D_{j}$) to calculate the percentage represented. The constraint sums all arcs $x_{ij}$ and obtains a weighted average distance.

$\sum_{ij}{\left(\frac{d_{ij}x_{ij}}{\sum_{j} D_j}\right)\le}\ \phi$

In [ ]:
phi = 0.5
model.addConstr(quicksum(x_ij[i,j]*dist[i,j] for i in id_supply_points for j in id_shelters)/D_j.sum()<=phi)

In addition, the following expression involves another distance constraint. The right side of this constraint is an input value $\varphi$, this value is defined as the minimum percentage of demand that must be met within a minimum distance value; the left side of this constraint includes an $x_{ij}$ arc multiplied by a new input data, a constant $a_{ij}$. This value is equal to 1 if a shelter $j$ served by a supply point $i$ is within a given distance value, and it is equal to 0 otherwise. For example, consider a network with collection centers and shelters, where the first are the supply points and the second are the demand points, and a value of 50 miles is considered to determine $a_{ij}$ and the value of $\varphi$ is 80%. Then, if a collection center $i$ supplies a shelter $j$, $a_{ij}$ will have the value of 1, if the distance between $i$ and $j$ is less than or equal to 50 miles; otherwise, it will be 0. The constraint will only consider those combinations that are within 50 miles and divide that value by the total demand. Therefore, the percentage of total demand that is within 50 miles must be greater than or equal to a minimum percentage value $\varphi$, which in this case is 80%. In other words, 80% of the demand supplied must be at a distance less than or equal to 50 miles. This constraint is very important to ensure a short distance on humanitarian supply trajectories.

$\sum_{ij}\left(\frac{a_{ij}x_{ij}}{\sum_{j} D_j}\right)\geq\ \varphi$

In [ ]:
varphi = 0.85
a = {(i,j):1 if dist[(i,j)]<=0.5 else 0 for i in id_supply_points for j in id_shelters}
model.addConstr(quicksum(x_ij[i,j]*a[i,j] for i in id_supply_points for j in id_shelters)/D_j.sum()>=varphi)

Furthermore, it is necessary to have a restriction that limits the attention in the supply points. While a facility may have enough capacity, this feature may not be conducive to providing quick attention. If a humanitarian attention facility is responsible for several shelters, the waiting rate for attention may increase considerably per person. Similarly, storage systems depend on the volume of people sheltered and the frequency with which demand is supplied to this community. An example of this can be illustrated if a person requires 15 liters of water per day for basic needs and has a water supply tank with a capacity of 10,000 liters, but with a current base of 7.5 liters per minute. Thus, this tank could supply water to over 650 people, but it would take more than 21 hours, which is totally counterproductive. Therefore, it is recommended to establish a value to limit the maximum flow of attention ($\delta$). It is important to mention that this constraint may not be indispensable for some case studies, and then the user can run the model without it.

$x_{ij}\ \le\delta\ \ \ \forall ij$

In [ ]:
delta = 3750

model.addConstrs(x_ij[i,j] <= delta for i in id_supply_points for j in id_shelters)

Two constraints were introduced in the model, these limit the number of supply points open. One is setting a minimum value $F_{min}$ and the other an upper bound, $F_{max}$. These values are the number of supply points that could possibly be opened:

$\sum_{i=1}^{n}B_i\le\ F_{max}\ \ ;\sum_{i=1}^{n}B_i\ \geq F_{min}$

In [ ]:
Fmin = 1
Fmax = 140

model.addConstr(quicksum(B_i[i] for i in id_supply_points) >= Fmin)
model.addConstr(quicksum(B_i[i] for i in id_supply_points) <= Fmax)

In [ ]:
model.optimize()

##### The optimal plan is as follows:

In [ ]:
if model.status==2:
    print("Objective function: ", str(round(model.objVal,2)))
    for v in model.getVars():
        if v.x>0:
            print(str(v.VarName)+"="+str(round(v.x,2)))
else:
     print("Infeasible solution")   

Therefore, the total cost of the operation was approximately 10,533 USD, with 12 tanks or supply nodes supplying 12 shelters. The results were favorable, not only because an optimal response was obtained that met all the constraints described at the beginning of this section, but also because 81.2% of the demand was supplied from a distance less than or equal to 0.5 km. This guarantees an effective response in the humanitarian operation.